# Import packages

In [1]:
# import system
from glob import glob
import csv
import os
from random import seed, uniform
from string import ascii_uppercase
import json
import requests, zipfile, io

# import numpy
import numpy as np

In [2]:
def norm(x1,x2):
    dx = x1[0] - x2[0]
    dy = x1[1] - x2[1]
    return (dx**2 + dy**2)**0.5

In [3]:
# for reproducibility
random_state = 912

# Download data from url

In [4]:
response = requests.get('https://www.sintef.no/contentassets/1338af68996841d3922bc8e87adc430c/pdp_100.zip')
print('Download Status:',response.ok)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall('./')

Download Status: True


# Processing all files in the dataset

In [5]:
# list of files in the Li & Lim benchmark
# https://www.sintef.no/projectweb/top/pdptw/li-lim-benchmark/
file_list = sorted(glob('pdp_100/*.txt'))
print(file_list)
print('\nBegin processing {} original datasets.'.format(len(file_list)))

['pdp_100/lc101.txt', 'pdp_100/lc102.txt', 'pdp_100/lc103.txt', 'pdp_100/lc104.txt', 'pdp_100/lc105.txt', 'pdp_100/lc106.txt', 'pdp_100/lc107.txt', 'pdp_100/lc108.txt', 'pdp_100/lc109.txt', 'pdp_100/lc201.txt', 'pdp_100/lc202.txt', 'pdp_100/lc203.txt', 'pdp_100/lc204.txt', 'pdp_100/lc205.txt', 'pdp_100/lc206.txt', 'pdp_100/lc207.txt', 'pdp_100/lc208.txt', 'pdp_100/lr101.txt', 'pdp_100/lr102.txt', 'pdp_100/lr103.txt', 'pdp_100/lr104.txt', 'pdp_100/lr105.txt', 'pdp_100/lr106.txt', 'pdp_100/lr107.txt', 'pdp_100/lr108.txt', 'pdp_100/lr109.txt', 'pdp_100/lr110.txt', 'pdp_100/lr111.txt', 'pdp_100/lr112.txt', 'pdp_100/lr201.txt', 'pdp_100/lr202.txt', 'pdp_100/lr203.txt', 'pdp_100/lr204.txt', 'pdp_100/lr205.txt', 'pdp_100/lr206.txt', 'pdp_100/lr207.txt', 'pdp_100/lr208.txt', 'pdp_100/lr209.txt', 'pdp_100/lr210.txt', 'pdp_100/lr211.txt', 'pdp_100/lrc101.txt', 'pdp_100/lrc102.txt', 'pdp_100/lrc103.txt', 'pdp_100/lrc104.txt', 'pdp_100/lrc105.txt', 'pdp_100/lrc106.txt', 'pdp_100/lrc107.txt', 'pdp_

# Data processing

In [6]:
def write_data(num_of_nodes,time_window):
    # set seed
    seed(random_state)
    np.random.seed(seed=random_state)
    # check the parameters
    assert num_of_nodes % 2 == 0
    assert time_window == True or time_window == False
    
    num_of_customer = int(num_of_nodes / 2)
    double_letter_set = ['{}{}'.format(ascii_uppercase[i],ascii_uppercase[j]) for i in range(26) for j in range(26)]

    for n,file_name in enumerate(file_list):
        pickup, delivery = [], []

        ''' get information from the datafile
        '''
        with open(file_name,'r') as f:
            for i,line in enumerate(f):
                # skip the first two lines
                if i == 0: 
                    speed = float(line.replace('\n','').split('\t')[-1])
                    # apprently some files have speed = 0
                    speed = speed if speed != 0 else 1
                if i <= 1: continue
                # read lines until pick-up and dilvery information is collected
                cache = [float(piece) for piece in line.replace('\n','').split('\t')]
                # if delivery
                if cache[-1] == 0 and len(delivery) < num_of_customer:
                    delivery.append(cache)
                elif cache[-2] == 0 and len(pickup) < num_of_customer:
                    pickup.append(cache)
                if len(pickup) == num_of_customer and len(delivery) == num_of_customer:
                    break

        ''' start and end depot, and other depots
        '''
        x_range = [case[1] for case in pickup+delivery]
        y_range = [case[2] for case in pickup+delivery]
        x_min, x_max = min(x_range), max(x_range)
        y_min, y_max = min(y_range), max(y_range)
        start_depot = [(uniform(x_min,x_max),uniform(y_min,y_max)) for _ in range(num_of_customer)]
        end_depot = [(uniform(x_min,x_max),uniform(y_min,y_max)) for _ in range(num_of_customer)]

        pickup_depot = [(case[1],case[2]) for case in pickup]
        delivery_depot = [(case[1],case[2]) for case in delivery]
        all_depots = pickup_depot + delivery_depot + start_depot + end_depot

        ''' vehicle capacity
        '''
        mean_demand = np.mean([case[3] for case in pickup])
        max_demand = max([case[3] for case in pickup])
        min_demand = min([case[3] for case in pickup])
        capacity = [max(np.random.normal(1*mean_demand,0.4*mean_demand),min_demand) for _ in range(num_of_customer)]
        # fix the last one to be max demand
        capacity[-1] = max_demand

        ''' cost factor for each vehicle
        '''
        cost_factor = sorted([20*np.random.normal(1,0.2) for _ in range(num_of_customer)])
        correct_order = sorted(range(num_of_customer), key = capacity.__getitem__)
        cost_factor = [cost_factor[indices] for indices in correct_order]

        ''' get the timewindow to half
        '''
        for _ in range(num_of_customer):
            pickup[_][4] /= 2
            pickup[_][5] /= 2
            delivery[_][4] /= 2
            delivery[_][5] /= 2

        ''' write to .dat file
        '''
        folder_name = './processed_data_{}_nodes_no_time_window/'.format(num_of_nodes) if not time_window \
                        else './processed_data_{}_nodes_yes_time_window/'.format(num_of_nodes)
        
        os.makedirs(folder_name,exist_ok=True)
        
        with open(folder_name+os.path.split(file_name)[-1].replace('.txt','.dat'),'w') as writer:
            if n == len(file_list)-1: print('Writing data to:',folder_name)
            # set of all nodes
            writer.write('set N := ')
            writer.write(' '.join(double_letter_set[:4*num_of_customer]))
            writer.write(';\n\n')
            # set of transhipment nodes, exclude start and end depot
            writer.write('set T := ')
            writer.write(' '.join(double_letter_set[:2*num_of_customer]))
            writer.write(';\n\n')
            # vehicle data
            writer.write('table\tK={K}\tu(K)\to(K)\to_(K):\n')
            writer.write('\tK\tu\to\to_\t:=\n')
            for k in range(num_of_customer):
                writer.write('\t{:d}\t{:.2f}\t{:}\t{:}\n'
                             .format(k+1,capacity[k],double_letter_set[2*num_of_customer+2*k],double_letter_set[2*num_of_customer+2*k+1]))
            writer.write(';\n\n')
            # cost data
            writer.write('param c default 0 :=\n')
            for i in range(4*num_of_customer):
                for j in range(4*num_of_customer):
                    if i == j: continue
                    # if both are between start and end depots, then continue
                    if i >= 2*num_of_customer and j >= 2*num_of_customer: continue
                    for k in range(num_of_customer):
                        distance = norm(all_depots[i],all_depots[j])
                        writer.write('{:}\t{:}\t{:d}\t{:.2f}\n'
                                     .format(double_letter_set[i],double_letter_set[j],k+1,distance*cost_factor[k]))
            writer.write(';\n\n')

            # time data
            if time_window:
                writer.write('param tau default 0 :=\n')
                for i in range(4*num_of_customer):
                    for j in range(4*num_of_customer):
                        if i == j: continue
                        if i >= 2*num_of_customer and j >= 2*num_of_customer: continue
                        for k in range(num_of_customer):
                            distance = norm(all_depots[i],all_depots[j])
                            writer.write('{:}\t{:}\t{:d}\t{:.2f}\n'
                                         .format(double_letter_set[i],double_letter_set[j],k+1,distance/speed))
                writer.write(';\n\n')

            # request data
            demand = [case[3] for case in pickup]
            writer.write('table R={R}  q(R)  p(R)  d(R):\n')
            writer.write('\tR\tq\tp\td\t:=\n')
            for i in range(num_of_customer):
                writer.write('\t{:}\t{:}\t{:}\t{:}\n'
                             .format(i+1,demand[i],double_letter_set[i],double_letter_set[i+num_of_customer]))
            writer.write(';\n\n')
            
            # request time data
            min_tranport_time = [norm(all_depots[j],all_depots[j+num_of_customer])/speed \
                      for j in range(num_of_customer)]
            # we need to do a check if the time window is consistent (min time is left)
            arrival_time = [case[4] for case in pickup]\
            + [max(case[4],2*min_tranport_time[i]+pickup[i][4]) for i,case in enumerate(delivery)]
            depart_time = [case[5] for case in pickup]\
            + [max(case[5],2*min_tranport_time[i]+pickup[i][5]) for i,case in enumerate(delivery)]

            if time_window:
                writer.write('param : t_e t_l :=\n')
                for i in range(2*num_of_customer):
                    writer.write('\t{:}\t{:.2f}\t{:.2f}\t\n'
                                 .format(double_letter_set[i],arrival_time[i],depart_time[i]))
                writer.write(';\n\n')

        # write an extra pickle file to store the location in a dictionary
        location_dic = {}
        for i in range(4*num_of_customer):
            location_dic[double_letter_set[i]] = all_depots[i]

        with open(folder_name+os.path.split(file_name)[-1].replace('.txt','.json'), 'w') as handle:
            json.dump(location_dic, handle)

# Write 4 different sets of data

In [7]:
write_data(10,time_window=False)
write_data(10,time_window=True)
write_data(14,time_window=False)
write_data(14,time_window=True)

Writing data to: ./processed_data_10_nodes_no_time_window/
Writing data to: ./processed_data_10_nodes_yes_time_window/
Writing data to: ./processed_data_14_nodes_no_time_window/
Writing data to: ./processed_data_14_nodes_yes_time_window/
